# Real-Time Personalized News Recommendation
A complete Colab notebook with an interactive UI sidebar for recommended news articles.

In [1]:
# Install required packages
!pip install newsapi-python sentence-transformers pandas ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import os
import pandas as pd
import math
import numpy as np
from datetime import datetime, timezone
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

## Configuration
Set your NewsAPI key as an environment variable or directly below.

In [3]:
import os
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

# Option 1: set via environment variable in Colab
# os.environ['NEWSAPI_KEY'] = 'YOUR_KEY_HERE'

# Option 2: directly assign (less secure)
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Configuration
# Option 1: set via environment variable in Colab
# import os
# os.environ['NEWSAPI_KEY'] = 'YOUR_KEY_HERE'

# Option 2: directly assign (less secure)
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
import os
import pandas as pd
import math
import numpy as np
from datetime import datetime, timezone
from newsapi import NewsApiClient
from sentence_transformers import SentenceTransformer

# Configuration
NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '5dc3d16fe871435685839cc0e4a6da3f')
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Fetch articles
articles = []

# Top-headlines for sports & entertainment
for cat in ['sports', 'entertainment']:
    resp = newsapi.get_top_headlines(category=cat, language='en', page_size=50)
    for art in resp.get('articles', []):
        text = art.get('content') or art.get('description') or ''
        articles.append({
            'title': art.get('title'),
            'url': art.get('url'),
            'category': cat,
            'published_at': art.get('publishedAt'),
            'text': text
        })

# Politics via everything endpoint
resp = newsapi.get_everything(
    q='politics',
    language='en',
    page_size=50,
    sort_by='publishedAt'
)
for art in resp.get('articles', []):
    text = art.get('content') or art.get('description') or ''
    articles.append({
        'title': art.get('title'),
        'url': art.get('url'),
        'category': 'politics',
        'published_at': art.get('publishedAt'),
        'text': text
    })

# Create DataFrame
df = pd.DataFrame(articles)
df['published_at'] = pd.to_datetime(df['published_at'])
df['age_hours'] = (
    datetime.now(timezone.utc)
    - df['published_at']
).dt.total_seconds() / 3600

# Preview
df.head()


,title,url,category,published_at,text,age_hours
0,Yankees release Marcus Stroman in surprise mov...,https://nypost.com/2025/08/01/sports/yankees-r...,sports,2025-08-01 18:11:15+00:00,MIAMI The Yankees are done with Marcus Stroman...,26.183983
1,"Chargers-Lions offensive report card: 2nd, 3rd...",https://www.prideofdetroit.com/2025/8/1/244789...,sports,2025-08-01 16:14:45+00:00,The Detroit Lions exhibition season opened wit...,28.125650
2,Generational swimming talents Summer McIntosh ...,https://www.nbcnews.com/sports/swimming/summer...,sports,2025-08-01 15:34:10+00:00,In one of the most consequential non-Olympic r...,28.802039
3,Everyone gets a goal: Sounders hang seven on C...,https://www.sounderatheart.com/2025/08/everyon...,sports,2025-08-01 15:13:22+00:00,SEATTLE Facing the defending Concacaf Champion...,29.148706
4,It’s Official! The 2027 MLB All-Star Game is C...,https://www.bleachernation.com/cubs/2025/08/01...,sports,2025-08-01 14:48:52+00:00,"For the first time in 37 years, the Chicago Cu...",29.557039


## Fetch Articles
- Sports & Entertainment via top-headlines
- Politics via the everything endpoint

## Compute Embeddings

In [9]:
# Compute text embeddings
df['embedding'] = embedder.encode(
    df['text'].tolist(),
    normalize_embeddings=True
).tolist()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Build User Profile Embedding

In [10]:
# Define user category weights
user_weights = {'sports': 0.5, 'politics': 0.3, 'entertainment': 0.2}

# Compute centroids
centroids = {}
for cat, w in user_weights.items():
    embs = [emb for emb, c in zip(df['embedding'], df['category']) if c == cat]
    if embs:
        centroids[cat] = np.mean(embs, axis=0)

# Weighted average profile
user_profile = sum(w * centroids[cat] for cat, w in user_weights.items() if cat in centroids)

## Score and Select Top Articles

In [11]:
# Score articles
scores = []
for _, row in df.iterrows():
    cat_score = user_weights.get(row['category'], 0)
    freshness = math.exp(-0.1 * row['age_hours'])
    sim = np.dot(user_profile, row['embedding'])
    score = 0.5 * cat_score + 0.3 * freshness + 0.2 * sim
    scores.append(score)

df['score'] = scores
topk = df.sort_values('score', ascending=False).head(20).reset_index(drop=True)
topk[['title','category','published_at','score']]

,title,category,published_at,score
0,Yankees release Marcus Stroman in surprise mov...,sports,2025-08-01 18:11:15+00:00,0.302536
1,Tim Brando ranks Top 15 college football teams...,sports,2025-08-01 14:18:26+00:00,0.299192
2,MLB trade deadline grades for every team: Yank...,sports,2025-08-01 14:04:00+00:00,0.297758
3,Everyone gets a goal: Sounders hang seven on C...,sports,2025-08-01 15:13:22+00:00,0.294901
4,"Chargers-Lions offensive report card: 2nd, 3rd...",sports,2025-08-01 16:14:45+00:00,0.294154
5,Patriots pursuing Terry McLaurin would make se...,sports,2025-08-01 11:44:01+00:00,0.293852
6,Pirates GM makes surprising admission about Da...,sports,2025-08-01 13:20:00+00:00,0.293747
7,45 Thoughts on the Patriots Through Eight Trai...,sports,2025-08-01 11:01:20+00:00,0.292618
8,"Peter Schmuck: Turns out, Orioles EVP/GM Mike ...",sports,2025-08-01 10:00:11+00:00,0.291691
9,"11 players later, the Twins dominated the trad...",sports,2025-08-01 11:01:34+00:00,0.291174


## Interactive UI Sidebar

In [12]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Create buttons for each recommended article
buttons = []
for idx, row in topk.iterrows():
    btn = widgets.Button(description=row['title'], layout=widgets.Layout(width='auto'))
    buttons.append(btn)

# Sidebar and content panes
sidebar = widgets.VBox(buttons, layout=widgets.Layout(width='30%', height='600px', overflow='auto'))
content = widgets.Output(layout=widgets.Layout(width='70%', padding='10px'))

# Button click handler
def on_button_click(btn):
    idx = buttons.index(btn)
    art = topk.iloc[idx]
    content.clear_output()
    with content:
        display(HTML(f"<h2>{art['title']}</h2>"))
        display(HTML(f"<p><em>{art['category']} | {art['published_at']}</em></p>"))
        display(HTML(f"<p>{art['text']}</p>"))
        display(HTML(f"<p><a href='{art['url']}' target='_blank'>Read full article</a></p>"))

for btn in buttons:
    btn.on_click(on_button_click)

# Display the UI
display(widgets.HBox([sidebar, content]))

## Share via Binder + Voilà
1. Push this notebook to GitHub in a public repo.
2. Include `requirements.txt` with:
```
newsapi-python
sentence-transformers
pandas
ipywidgets
```
3. Use the Binder URL:
```
https://mybinder.org/v2/gh/<YourUser>/<YourRepo>/main?urlpath=voila/render/<NotebookName>.ipynb
```